In [3]:
!git clone https://github.com/Lightning-AI/lit-gpt

Cloning into 'lit-gpt'...
remote: Enumerating objects: 5300, done.
remote: Counting objects: 100% (1517/1517), done.
remote: Compressing objects: 100% (420/420), done.
remote: Total 5300 (delta 1307), reused 1170 (delta 1094), pack-reused 3783
Receiving objects: 100% (5300/5300), 1.81 MiB | 7.13 MiB/s, done.
Resolving deltas: 100% (3626/3626), done.


In [5]:
cd lit-gpt

[Errno 2] No such file or directory: 'lit-gpt'
/workspace/lit-gpt


In [6]:
pwd

'/workspace/lit-gpt'

In [7]:
!pip install -r requirements-all.txt

  Cloning https://github.com/Lightning-AI/lightning (to revision 6cbe9ceb560d798892bdae9186291acf9bf5d2e3) to /tmp/pip-install-rybc7ofv/lightning_5157331daff2408cbdd959ed3fae6ae3
  Running command git clone --filter=blob:none --quiet https://github.com/Lightning-AI/lightning /tmp/pip-install-rybc7ofv/lightning_5157331daff2408cbdd959ed3fae6ae3
  Running command git rev-parse -q --verify 'sha^6cbe9ceb560d798892bdae9186291acf9bf5d2e3'
  Running command git fetch -q https://github.com/Lightning-AI/lightning 6cbe9ceb560d798892bdae9186291acf9bf5d2e3
  Running command git checkout -q 6cbe9ceb560d798892bdae9186291acf9bf5d2e3
  Resolved https://github.com/Lightning-AI/lightning to commit 6cbe9ceb560d798892bdae9186291acf9bf5d2e3
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/EleutherAI/lm-evaluation-harness.git (to revis

In [8]:
!python scripts/download.py --repo_id TinyLlama/TinyLlama-1.1B-intermediate-step-955k-token-2T

tokenizer_config.json: 100%|███████████████████| 776/776 [00:00<00:00, 8.99MB/s]

pytorch_model.bin:   0%|                            | 0.00/4.40G [00:00<?, ?B/s]

tokenizer.json:   0%|                               | 0.00/1.84M [00:00<?, ?B/s]


tokenizer.model: 100%|███████████████████████| 500k/500k [00:00<00:00, 19.6MB/s]



Fetching 5 files:  20%|█████▍                     | 1/5 [00:01<00:04,  1.18s/it]
pytorch_model.bin:   0%|                   | 10.5M/4.40G [00:00<02:20, 31.3MB/s]
pytorch_model.bin:   0%|                   | 21.0M/4.40G [00:00<01:35, 45.7MB/s]

tokenizer.json: 100%|██████████████████████| 1.84M/1.84M [00:00<00:00, 3.68MB/s]

pytorch_model.bin:   1%|▏                  | 31.5M/4.40G [00:00<01:22, 52.9MB/s]
pytorch_model.bin:   1%|▏                  | 41.9M/4.40G [00:00<01:19, 55.1MB/s]
pytorch_model.bin:   1%|▏                  | 52.4M/4.40G [00:01<01:16, 56.9MB/s]
pytorch_model.bin:   1%|▎                  | 62.9M/4.40G [00:01<01:13, 58.8MB/s]
pytorch_model.bin: 

In [9]:
!python scripts/convert_hf_checkpoint.py --checkpoint_dir checkpoints/TinyLlama/TinyLlama-1.1B-intermediate-step-955k-token-2T

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
Model config {'name': 'tiny-llama-1.1b', 'hf_config': {'org': 'TinyLlama', 'name': 'TinyLlama-1.1B-intermediate-step-955k-token-2T'}, 'block_size': 2048, 'vocab_size': 32000, 'padding_multiple': 64, 'padded_vocab_size': 32000, 'n_layer': 22, 'n_head': 32, 'n_embd': 2048, 'rotary_percentage': 1.0, 'parallel_residual': False, 'bias': False, 'lm_head_bias': False, 'n_query_groups': 4, 'shared_attention_norm': False, '_norm_class': 'RMSNorm', 'norm_eps': 1e-05, '_mlp_class': 'LLaMAMLP', 'gelu_approximate': 'none', 'intermediate_size': 5632, 'rope_condense_ratio': 1, 'rope_base': 10000}
Processing checkpoints/TinyLlama/TinyLlama-1.1B-intermediate-step-955k-token-2T/pytorch_model.bin
Loading 'lm_head.weight' into RAM
Loading 'model.embed_tokens.

In [10]:
!python scripts/prepare_alpaca.py --checkpoint_dir checkpoints/TinyLlama/TinyLlama-1.1B-intermediate-step-955k-token-2T

Loading data file...
Loading tokenizer...
train has 2,643 samples
test has 106 samples
Processing train split ...
100%|██████████████████████████████████████| 2643/2643 [00:02<00:00, 905.91it/s]
Processing test split ...
100%|████████████████████████████████████████| 106/106 [00:00<00:00, 900.10it/s]


In [18]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

0

In [19]:
!python finetune/lora.py --checkpoint_dir checkpoints/TinyLlama/TinyLlama-1.1B-intermediate-step-955k-token-2T --precision bf16-true --quantize bnb.nf4

{'eval_interval': 100, 'save_interval': 100, 'eval_iters': 100, 'eval_max_new_tokens': 100, 'log_interval': 1, 'devices': 1, 'learning_rate': 0.0003, 'batch_size': 128, 'micro_batch_size': 4, 'gradient_accumulation_iters': 32, 'max_iters': 50000, 'weight_decay': 0.01, 'lora_r': 8, 'lora_alpha': 16, 'lora_dropout': 0.05, 'lora_query': True, 'lora_key': False, 'lora_value': True, 'lora_projection': False, 'lora_mlp': False, 'lora_head': False, 'warmup_steps': 100}
Seed set to 1337
Loading model 'checkpoints/TinyLlama/TinyLlama-1.1B-intermediate-step-955k-token-2T/lit_model.pth' with {'name': 'tiny-llama-1.1b', 'hf_config': {'org': 'TinyLlama', 'name': 'TinyLlama-1.1B-intermediate-step-955k-token-2T'}, 'block_size': 2048, 'vocab_size': 32000, 'padding_multiple': 64, 'padded_vocab_size': 32000, 'n_layer': 22, 'n_head': 32, 'n_embd': 2048, 'rotary_percentage': 1.0, 'parallel_residual': False, 'bias': False, 'lm_head_bias': False, 'n_query_groups': 4, 'shared_attention_norm': False, '_norm_c